In [5]:
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from shapely.geometry import MultiLineString
import polyline
import geopandas as gpd

load_dotenv()
api_key = os.getenv("google_api")
gmaps = googlemaps.Client(key=api_key)
GDRIVE_FOLDER = Path(os.getenv("GDRIVE_FOLDER"))
origins = GDRIVE_FOLDER / "origin.csv"
destinations = GDRIVE_FOLDER / "attractions.csv"
origin_df = pd.read_csv(origins)
destinations_df = pd.read_csv(destinations)
now = datetime.now()


origin = (
    39.946196,
    -75.139832,
)  # hardcoded here since my origin doesn't change, but if yours does you can tweak the function to accept origins.


def distance_duration_iteration(mode):
    """returns a list of origin/destination dicts for driving or transit, respectively"""
    driving_list = []
    transit_list = []
    
    def distance_duration(destination):
        output = gmaps.directions(
            origin,
            destination,
            mode,
            units="imperial",
            departure_time=now,
        )
        output.append(name)

        if mode == "driving":
            driving_list.append(output)

        if mode == "transit":
            transit_list.append(output)

    """loops through the destinations file and runs the distance duration function"""
    for idx, row in destinations_df.iterrows():
        lat = row["Latitude"]
        lon = row["Longitude"]
        name = row["Name"]
        destination = (lat, lon)
        distance_duration(destination)
        

    if mode == "driving":
        print("returning driving list")
        return driving_list
    if mode == "transit":
        print("returning transit list")
        return transit_list


def save_list_in_memory(transpo_list):
    '''saves list from distance_duration_iteration into memory to avoid repeated calls to the API'''
    transpo_list = transpo_list
    return transpo_list

def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the google api returns"""
    labels = ["distance", "duration", "lat", "lng"]
    df = pd.DataFrame(columns=labels)

    try:
        for item in list_of_dicts:
            destination_lat = item[0]["legs"][0]["end_location"]["lat"]
            destination_lng = item[0]["legs"][0]["end_location"]["lng"]
            distance = item[0]["legs"][0]["distance"]["text"]
            duration = item[0]["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
    except:
        for item in list_of_dicts[0]:
            destination_lat = item["legs"][0]["end_location"]["lat"]
            destination_lng = item["legs"][0]["end_location"]["lng"]
            distance = item["legs"][0]["distance"]["text"]
            duration = item["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)

    return df


def df_to_csv(df):
    df.to_csv(GDRIVE_FOLDER / f"{mode}_travel_times.csv", sep=",")

In [6]:
driving_list = save_list_in_memory(distance_duration_iteration("driving"))
transit_list = save_list_in_memory(distance_duration_iteration("transit"))

returning driving list
returning transit list


In [7]:
print(driving_list[1])

[{'bounds': {'northeast': {'lat': 39.9499809, 'lng': -75.10153679999999}, 'southwest': {'lat': 39.835266, 'lng': -75.1588192}}, 'copyrights': 'Map data ©2022 Google', 'legs': [{'distance': {'text': '13.6 mi', 'value': 21861}, 'duration': {'text': '23 mins', 'value': 1358}, 'duration_in_traffic': {'text': '22 mins', 'value': 1308}, 'end_address': '164 S Broad St, Woodbury, NJ 08096, USA', 'end_location': {'lat': 39.835266, 'lng': -75.1550364}, 'start_address': '211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA', 'start_location': {'lat': 39.9464869, 'lng': -75.1399589}, 'steps': [{'distance': {'text': '223 ft', 'value': 68}, 'duration': {'text': '1 min', 'value': 16}, 'end_location': {'lat': 39.9467392, 'lng': -75.1405688}, 'html_instructions': 'Head <b>north</b>', 'polyline': {'points': 'q`yrFvwriMACAAC?C?A?A@A?ABCFCFADADQ|A'}, 'start_location': {'lat': 39.9464869, 'lng': -75.1399589}, 'travel_mode': 'DRIVING'}, {'distance': {'text': '131 ft', 'value': 40}, 'duration': {'tex

In [26]:
def unpack_geometries(transpo_list):
    """accepts either the transit list or the driving list from the distance_duration_iteration function and parses accordingly"""
    polylines = []

    if transpo_list == driving_list: 
        mode = "driving"
    if transpo_list == transit_list:
        mode = "transit" # necessary to set mode for f string at end of function


    for item in transpo_list:
        if not item:
            pass  # just appending blank items here if there's no polyline
        else:
            overview_line = [
                polyline.decode(
                    item[0]["overview_polyline"]["points"], geojson=True
                )
            ]
            line = MultiLineString(overview_line)
            polylines.append(line)

    df = pd.DataFrame(polylines)

    print(df)

    df.columns = ["strings"]
    gdf = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="strings")
    gdf.to_file(GDRIVE_FOLDER / f"{mode}_polylines.geojson", driver="GeoJSON")

In [ ]:
detailed_linestrings_by_leg_of_trip = []
travel_mode = []

for item in transit_list:
    if not item:
        pass
    else:
        line = [polyline.decode(item[0]["legs"][0]["steps"][0]["steps"][0]["polyline"]["points"], geojson= True)]
        multiline = MultiLineString(line)
        detailed_linestrings_by_leg_of_trip.append(multiline)
        mode =  item[0]["legs"][0]["steps"][0]["steps"][0]["travel_mode"]
        travel_mode.append(mode)
     
df = pd.DataFrame(detailed_linestrings_by_leg_of_trip)
df["mode"] = travel_mode
print(df.head(30))
# if __name__ == "__main__":
#     #holds lists in memory so api isn't repeatedly called 
#     driving_list = save_list_in_memory(distance_duration_iteration("driving"))
#     transit_list = save_list_in_memory(distance_duration_iteration("transit"))
#     #runs all functions for driving times/durations/geometries
#     unpacked_driving = unpack_dicts(distance_duration_iteration("driving"))
#     df_to_csv(unpacked_driving)
#     unpack_geometries(driving_list)
#     #runs the same for transit
#     unpacked_transit = unpack_dicts(distance_duration_iteration("transit"))
#     df_to_csv(unpacked_transit)
#     unpack_geometries(transit_list)
    